In [44]:
import helper_functions as hf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from matplotlib.colors import to_rgb, to_hex

def lighten_color(hex_color: str, factor: float) -> str:
    """Lightens a hex color by blending it with white."""
    rgb = to_rgb(hex_color)
    lighter_rgb = [(1 - (1 - c) * (1 - factor)) for c in rgb]
    return to_hex(lighter_rgb)  # type: ignore

BASE_PATH = hf.FpathRegistry.get_path_hidden().joinpath("schedules")

def get_subdf_for_sport(sport: str) -> pd.DataFrame:
    players = hf.DATA_NOW.players.fillna("").set_index("nickname", drop=False)
    players = players[players[f"subteam_{sport}"].str.isnumeric() | (players[f"subteam_{sport}"] == "R")]
    HIDDEN_DF = hf.FpathRegistry.get_hidden_responses().set_index("nickname", drop=False)
    players = players.join(HIDDEN_DF[[col for col in HIDDEN_DF.columns if not col in players.columns]], how="left")
    return players


def plot_matchup_table(table_data: list, ax: Axes, fontsize: int = 12) -> None:
    ax.axis("off")
    table = ax.table(
        cellText=table_data,
        colLabels=None,
        cellLoc="left",
        loc="center"
    )
    table.auto_set_font_size(False)
    table.set_fontsize(fontsize)
    table.auto_set_column_width(col=list(range(len(table_data[0]))))
    
    # Add padding and alignment for each cell
    for key, cell in table.get_celld().items():
        cell.PAD = 0.1
        cell.set_height(cell.get_height() + 0.025)
        if key[0] == 0:  # Header row
            cell.set_text_props(weight="bold")
        cell_text = cell.get_text().get_text()
        if cell_text.startswith("A: "):
            cell.set_facecolor(lighten_color(hf.Team.colors[0], 0.7))
        if cell_text.startswith("B: "):
            cell.set_facecolor(lighten_color(hf.Team.colors[1], 0.7))
        if cell_text.startswith("C: "):
            cell.set_facecolor(lighten_color(hf.Team.colors[2], 0.7))
        # For the numeric ones, use shades of gray
        for i in range(5):
            if cell_text.endswith(str(i)) and len(cell_text) < 3:
                cell.set_facecolor(lighten_color("#808080", 0.2 * i))
                break


In [ ]:
match_df = hf.DATA_NOW.match_df

ping_pong = match_df[match_df["sport"] == "ping_pong"]
days = np.unique(ping_pong["day"])

# Create a PDF to save the A4 pages
table_dict = {"1": "MPE Table", "2": "IPP Table 1", "3": "IPP Table 2"}


table = "1"
table_name = table_dict["1"]
fpath = BASE_PATH.joinpath(f"ping_pong_matches_{table}.pdf")
ping_pong = ping_pong[ping_pong["location"] == table]

with PdfPages(fpath) as pdf:
    fig, axes = plt.subplots(5, 1, figsize=(8.27, 11.69))  # A4 size in inches (210mm x 297mm)
    for ax, day in zip(axes, ["monday", "tuesday", "wednesday", "thursday", "friday"]):
        subdf = ping_pong[ping_pong["day"] == day.capitalize()].sort_values("time")
        table_data = [["Match", "Time", "Player 1", "Player 2", "Result"]]
        for i, match in subdf.iterrows():
            table_data.append([
                match['full_key'].replace("ping_pong_", ""),
                match["time"],
                match["team_a"],
                match["team_b"],
                ""
            ])
        plot_matchup_table(table_data, ax)
        ax.set_title(day.capitalize(), fontsize=14, weight="bold", pad=len(subdf))
    fig.subplots_adjust(top=0.94, bottom=0.02, left=0.1, right=0.9, hspace=0.4)
    fig.suptitle(f"Ping Pong Matches at {table_name}", fontsize=16, weight="bold")
    pdf.savefig(fig)
    plt.close(fig)


In [ ]:
# Create Printout for Running/Sprints:
players = hf.DATA_NOW.players.fillna("").set_index("nickname", drop=False)
players = players[players["subteam_running_sprints"].str.isnumeric()]
HIDDEN_DF = hf.FpathRegistry.get_hidden_responses().set_index("nickname", drop=False)
players = players.join(HIDDEN_DF[[col for col in HIDDEN_DF.columns if not col in players.columns]], how="left")
players[["nickname", "name", "Team"]]
# Create a PDF to save the A4 pages
fname = BASE_PATH.joinpath("running_sprints.pdf")

with PdfPages(fname) as pdf:
    fig, axes = plt.subplots(3, 1, figsize=(8.27, 11.69))  # A4 size in inches (210mm x 297mm)
    fig.suptitle("Running/Sprints", fontsize=16, weight="bold")
    for team in "ABC":
        subdf = players[players["Team"] == f"Team {team}"].sort_values("name")
        ax = axes[ord(team) - ord("A")]
        ax.axis("off") 
        table_data = [["Nickname", "Name", "Team"]]
        for i, player in subdf.iterrows():
            table_data.append([
                player["nickname"],
                player["name"],
                player["Team"]
            ])
        
        # Create the table
        table = ax.table(
            cellText=table_data,
            colLabels=None,
            cellLoc="left",
            loc="center"
        )
        for key, cell in table.get_celld().items():
            cell.PAD = 0.1
            cell.set_height(cell.get_height() + 0.025)
            if key[0] == 0:  # Header row
                cell.set_text_props(weight="bold")
        table.auto_set_font_size(False)
        table.set_fontsize(12)
        table.auto_set_column_width(col=list(range(len(table_data[0]))))
    fig.subplots_adjust(top=0.94, bottom=0.02, left=0.1, right=0.9, hspace=0.4)
    fig.suptitle(f"Running/Sprints", fontsize=16, weight="bold")
    pdf.savefig(fig)
    plt.close(fig)


In [60]:
# Create Printout for any team sport event (Football, Volleyball, Basketball):


def get_figs_for_teamsport(sport: str) -> tuple[Figure, Figure]:
    players = get_subdf_for_sport(sport)
    matches = hf.DATA_NOW.match_df
    matches = matches[matches["sport"] == sport].sort_values(["time", "location"])
    sport_subkey = f"subteam_{sport}"
    # Generate a table for the matches:
    table_data = [["Match", "Time", "Loc", "Team 1", "Team 2", "Result"]]
    for i, match in matches.iterrows():
        table_data.append([
            match['full_key'].replace(f"{sport}_", ""),
            match["time"],
            match["location"],
            match["team_a"],
            match["team_b"],
            ""
        ])
    fig, axes = plt.subplots(2, 1, figsize=(8.27, 11.69))
    fig2, axes2 = plt.subplots(2, 1, figsize=(8.27, 11.69))
    plot_matchup_table(table_data, axes[0])
    axes[0].set_title("Matches", fontsize=14, weight="bold", pad=len(matches)*1.1)
    for i, team in enumerate("ABC"):
        subdf = players[players["Team"] == f"Team {team}"].sort_values([sport_subkey, "name"])
        ax = axes[i + 1] if i == 0 else axes2[i - 1]
        table_data = [["Nickname", "Name", "Subteam", "Is there?"]]
        for i, player in subdf.iterrows():
            table_data.append([
                player["nickname"],
                player["name"],
                player["Team"][-1] + player[sport_subkey],
                ""
            ])
        for _ in range(2):
            table_data.append(["", "", "", ""])
        plot_matchup_table(table_data, ax, fontsize=7)
        ax.set_title(f"Team {team}", fontsize=14, weight="bold", pad=1)
    fig.subplots_adjust(top=0.94, bottom=0.06, left=0.1, right=0.9, hspace=0.2)
    fig2.subplots_adjust(top=0.94, bottom=0.06, left=0.1, right=0.9, hspace=0.2)
    sport_name = sport.replace("_", " ").capitalize()
    fig.suptitle(f"{sport_name}: Matches/Players", fontsize=16, weight="bold")
    fig2.suptitle(f"{sport_name}: Players, continued", fontsize=16, weight="bold")
    return fig, fig2


sports = ["football", "volleyball", "basketball", "capture_the_flag", "beer_pong", "badminton", "foosball", "spikeball"]
for sport in sports:
    fname = BASE_PATH.joinpath(f"{sport}.pdf")
    with PdfPages(fname) as pdf:
        fig1, fig2 = get_figs_for_teamsport(sport)
        pdf.savefig(fig1)
        plt.close(fig1)
        pdf.savefig(fig2)
        plt.close(fig2)



In [65]:
def get_fig_for_solo_sport(sport: str) -> Figure:
    hf.DATA_NOW.reload()
    players = get_subdf_for_sport(sport)
    # players = hf.DATA_NOW.players.fillna("").set_index("nickname", drop=False)
    matches = hf.DATA_NOW.match_df
    matches = matches[matches["sport"] == sport].sort_values(["time", "location"])
    
    table_data = [["Match", "Time", "Loc", "Player 1", "Player 2", "Result"]]
    for i, match in matches.iterrows():
        player_a = players[players["nickname"] == match["team_a"].split(": ")[1]].iloc[0]
        player_b = players[players["nickname"] == match["team_b"].split(": ")[1]].iloc[0]
        player_a_str = f"{player_a['nickname']}\n({player_a['name']})"
        player_b_str = f"{player_b['nickname']}\n({player_b['name']})"
        table_data.append([
            match['full_key'].replace(f"{sport}_", ""),
            match["time"],
            match["location"],
            player_a_str,
            player_b_str,
            ""
        ])
    fig, ax = plt.subplots(figsize=(8.27, 11.69))
    plot_matchup_table(table_data, ax)
    ax.set_title("Matches", fontsize=14, weight="bold", pad=len(matches)*1.1)   

    fig.subplots_adjust(top=0.94, bottom=0.06, left=0.1, right=0.9, hspace=0.2)
    sport_name = sport.replace("_", " ").capitalize()
    fig.suptitle(f"{sport_name}: Matches", fontsize=16, weight="bold")
    return fig


sports = ["tennis", "chess"]
for sport in sports:
    fname = BASE_PATH.joinpath(f"{sport}.pdf")
    with PdfPages(fname) as pdf:
        fig = get_fig_for_solo_sport(sport)
        pdf.savefig(fig)
        plt.close(fig)
